In [1]:
# Agents
from Agents.togetherAIAgent import generate_article_from_query
from Agents.wikiAgent import get_wiki_data
from Agents.rankerAgent import rankerAgent
from Query_Modification.QueryModification import query_Modifier, getKeywords

from Ranking.RRF.RRF_implementation import reciprocal_rank_fusion_three, reciprocal_rank_fusion_six, reciprocal_rank_fusion_multiple_lists

# Retrieval Models
from Retrieval.tf_idf import tf_idf_pipeline
from Retrieval.bm25 import bm25_pipeline
from Retrieval.vision import vision_pipeline
from Retrieval.openSource import open_source_pipeline
from Baseline.boolean import boolean_pipeline
# Answer Generation
from AnswerGeneration.getAnswer import generate_answer_withContext, generate_answer_zeroShot

import json

/home/yash/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yash/miniconda3/envs/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
miniWikiCollection = json.load(open('Datasets/mini_wiki_collection.json', 'r'))
miniWikiCollectionDict = {}

for wiki in miniWikiCollection:
    miniWikiCollectionDict[wiki['wikipedia_id']] = " ".join(wiki['text'])
    # miniWikiCollectionDict[wiki['wikipedia_id']] = " ".join((" ".join(wiki['text'])).split()[:500])

In [3]:
query = "Who is Harry Potter?"

In [4]:
query = "Who is Harry Potter?"
modified_query = query_Modifier(query)

# Context Generation
article = generate_article_from_query(query)

# Keyword Extraction and getting Context from Wiki
keywords = getKeywords(query)
wiki_data = get_wiki_data(keywords)

2024-11-19 07:00:08,945 - INFO - Wikipedia: language=en, user_agent: WikipediaSearcher/1.0 (Wikipedia-API/0.7.1; https://github.com/martin-majlis/Wikipedia-API/), extract_format=1
2024-11-19 07:00:08,946 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=info&titles=Harry Potter&inprop=protection|talkid|watched|watchers|visitingwatchers|notificationtimestamp|subjectid|url|readable|preload|displaytitle
2024-11-19 07:00:10,210 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=categories&titles=Harry Potter&cllimit=500
2024-11-19 07:00:10,506 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=extracts&titles=Harry Potter&explaintext=1&exsectionformat=wiki
2024-11-19 07:00:10,823 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Harry Potter&pllimit=500
2024-11-19 07:00:11,335 - INFO - Request URL: https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Harry Potter&plli

In [5]:
article

'Harry Potter: The Boy Who Lived\n\nHarry Potter is a fictional character created by British author J.K. Rowling. He is the main protagonist in a series of fantasy novels, films, and other media that have become a global phenomenon. The character\'s story has captivated audiences of all ages, making him one of the most beloved and iconic figures in popular culture.\n\nBackground and Creation\n\nHarry Potter was born on July 31, 1980, to James and Lily Potter, two wizards who were killed by the dark wizard Lord Voldemort when Harry was just a baby. This event, known as the "Killing Curse," left Harry with a lightning-shaped scar on his forehead and made him famous in the wizarding world as "The Boy Who Lived." Rowling drew inspiration for the character from her own childhood experiences, including her struggles with depression and her fascination with mythology and folklore.\n\nThe Harry Potter series follows the adventures of the young wizard as he attends Hogwarts School of Witchcraft

In [6]:
wiki_data[0]

"Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).\nThe series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States.  A series of many genres, including fantasy, drama, coming-of-age fiction, and the British school story (which includes elements of mystery, thriller, adventure, horror, and romance), the world of Harry Potter explores numerous themes and includes many cultural meanings and references. Major themes in the series include prejudice, corruption, madness, l

In [7]:
modified_query

'Who is Harry Potter, and what are his key characteristics and defining moments in the Harry Potter book series?\n'

In [8]:
boolean_ranking = boolean_pipeline(query)
tf_idf_ranking = tf_idf_pipeline(query)
bm25_ranking = bm25_pipeline(query)
vision_ranking = vision_pipeline(query)
open_source_ranking = open_source_pipeline(query)

boolean_ranking_modified = boolean_pipeline(modified_query)
tf_idf_ranking_modified = tf_idf_pipeline(modified_query)
bm25_ranking_modified = bm25_pipeline(modified_query)
vision_ranking_modified = vision_pipeline(modified_query)
open_source_ranking_modified = open_source_pipeline(modified_query)

tf_idf_bm25_open_RRF_Ranking = reciprocal_rank_fusion_three(tf_idf_ranking, bm25_ranking, open_source_ranking)
tf_idf_bm25_open_RRF_Ranking_modified = reciprocal_rank_fusion_three(tf_idf_ranking_modified, bm25_ranking_modified, open_source_ranking_modified)
tf_idf_bm25_open_RRF_Ranking_combined = reciprocal_rank_fusion_six(tf_idf_ranking, bm25_ranking, open_source_ranking, tf_idf_ranking_modified, bm25_ranking_modified, open_source_ranking_modified)

idf loaded...
tf-idf loaded...
vocab loaded...
document_matrix loaded...
ids loaded


100%|██████████| 1000/1000 [00:00<00:00, 33968.85it/s]


loaded embeddings


100%|██████████| 100/100 [00:00<00:00, 1370687.58it/s]


idf loaded...
tf-idf loaded...
vocab loaded...
document_matrix loaded...
ids loaded


100%|██████████| 1000/1000 [00:00<00:00, 33751.81it/s]


loaded embeddings


100%|██████████| 100/100 [00:00<00:00, 1067252.93it/s]


In [9]:
agent1_context = wiki_data[0]
agent2_context = article

boolean_context = miniWikiCollectionDict[boolean_ranking[0]]
tf_idf_context = miniWikiCollectionDict[tf_idf_ranking[0]]
bm25_context = miniWikiCollectionDict[str(bm25_ranking[0])]
vision_context = miniWikiCollectionDict[vision_ranking[0]]
open_source_context = miniWikiCollectionDict[open_source_ranking[0]]

boolean_context_modified = miniWikiCollectionDict[boolean_ranking_modified[0]]
tf_idf_context_modified = miniWikiCollectionDict[tf_idf_ranking_modified[0]]
bm25_context = miniWikiCollectionDict[str(bm25_ranking_modified[0])]
vision_context_modified = miniWikiCollectionDict[vision_ranking_modified[0]]
open_source_context_modified = miniWikiCollectionDict[open_source_ranking_modified[0]]

tf_idf_bm25_open_RRF_Ranking_context = miniWikiCollectionDict[tf_idf_bm25_open_RRF_Ranking[0]]
tf_idf_bm25_open_RRF_Ranking_modified_context = miniWikiCollectionDict[tf_idf_bm25_open_RRF_Ranking_modified[0]]
tf_idf_bm25_open_RRF_Ranking_combined_context = miniWikiCollectionDict[tf_idf_bm25_open_RRF_Ranking_combined[0]]

In [10]:
#Generating answers

agent1_answer = generate_answer_withContext(query, agent1_context)
agent2_answer = generate_answer_withContext(query, agent2_context)

boolean_answer = generate_answer_withContext(query, boolean_context)
tf_idf_answer = generate_answer_withContext(query, tf_idf_context)
bm25_answer = generate_answer_withContext(query, bm25_context)
vision_answer = generate_answer_withContext(query, vision_context)
open_source_answer = generate_answer_withContext(query, open_source_context)

boolean_answer_modified = generate_answer_withContext(modified_query, boolean_context_modified)
tf_idf_answer_modified = generate_answer_withContext(modified_query, tf_idf_context_modified)
bm25_answer_modified = generate_answer_withContext(modified_query, bm25_context)
vision_answer_modified = generate_answer_withContext(modified_query, vision_context_modified)
open_source_answer_modified = generate_answer_withContext(modified_query, open_source_context_modified)

tf_idf_bm25_open_RRF_Ranking_answer = generate_answer_withContext(query, tf_idf_bm25_open_RRF_Ranking_context)
tf_idf_bm25_open_RRF_Ranking_modified_answer = generate_answer_withContext(modified_query, tf_idf_bm25_open_RRF_Ranking_modified_context)
tf_idf_bm25_open_RRF_Ranking_combined_answer = generate_answer_withContext(query, tf_idf_bm25_open_RRF_Ranking_combined_context)

zeroShot = generate_answer_zeroShot(query)

In [11]:
# reRanker
rankerAgentInput = {
    "query": query,
    "agent1": agent1_answer,
    "agent2": agent2_answer,
    "boolean": boolean_answer,
    "tf_idf": tf_idf_answer,
    "bm25": bm25_answer,
    "vision": vision_answer,
    "open_source": open_source_answer,
    "boolean_modified": boolean_answer_modified,
    "tf_idf_modified": tf_idf_answer_modified,
    "bm25_modified": bm25_answer_modified,
    "vision_modified": vision_answer_modified,
    "open_source_modified": open_source_answer_modified,
    "tf_idf_bm25_open_RRF_Ranking": tf_idf_bm25_open_RRF_Ranking_answer,
    "tf_idf_bm25_open_RRF_Ranking_modified": tf_idf_bm25_open_RRF_Ranking_modified_answer,
    "tf_idf_bm25_open_RRF_Ranking_combined": tf_idf_bm25_open_RRF_Ranking_combined_answer,
    "zeroShot": zeroShot
}

best_model, best_answer = rankerAgent(rankerAgentInput)

In [12]:
best_model, best_answer

('tf_idf_bm25_open_RRF_Ranking_modified',
 "Harry Potter is the main protagonist of the series, a 17-year-old wizard who is known for his bravery, loyalty, and determination. He is an orphan who discovers he is famous in the wizarding world and begins attending Hogwarts School of Witchcraft and Wizardry, where he forms close friendships with Ron Weasley and Hermione Granger. Throughout the series, Harry faces numerous challenges as he battles against the dark wizard Lord Voldemort, who murdered Harry's parents and seeks to return to power. Key characteristics of Harry include his courage, selflessness, and willingness to risk himself to protect others. Defining moments in the series include his initial discovery of his true identity as a wizard, his battles against Voldemort, and his ultimate defeat of the Dark Lord in the final book.")